In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import os
import torch
import pandas as pd
import csv
from gensim.models.word2vec import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec
import logging
import re
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
info = pd.read_csv('/home/duyongkang/aminer/aminer.paper.info.csv')
auth_info = pd.read_csv('/home/duyongkang/aminer/aminer.author.expertise.tag.csv')

In [6]:
#先把全部学者编号和标签拿出来放到list中
ppr_list = info['paper_id'].values.tolist()             # 论文列表
text_list = info['text'].values.tolist()                # 文本列表
auth_list = auth_info['author_id'].values.tolist()      # 作者列表
exper_list = auth_info['expertise_tag'].values.tolist() # 专长列表

In [14]:
#去掉标点符号并分割成单词
lists_new = []
for list in exper_list:
#     string = re.sub("[\s+\.\;\!\/_,$%^)<}]*•≥=[:|{>(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", " ", list)
    string = re.sub("\W", " ", list)
    lists_new.append(string.split())

In [16]:
count_c = 0
for i in range(len(lists_new)):
    count_c += len(lists_new[i])
count_c

638643

In [15]:
lists_new[0]

['organization', 'quality', 'models']

In [4]:
# for i in range(len(lists_new)):
#     for j in range(len(lists_new[i])):
#         if( 'system' in lists_new[i][j]):
#             print('!')
#         test_str = re.search(r"\W",lists_new[i][j])
#         if test_str!=None:
#             print('i',i,' ','j',j,' ',lists_new[i][j])

In [5]:
# output_file = 'text.txt'
# with open(output_file,'w') as f:
#     for i in range(len(lists_new)):
#         outline = ''
#         for word in lists_new[i]:
#             word += ' '
#             outline += word
#         f.write(outline + "\n")
# f.close()

## 使用文本信息构建w2v模型

In [6]:
# class callback(CallbackAny2Vec):
#     '''Callback to print loss after each epoch.'''

#     def __init__(self):
#         self.epoch = 0

#     def on_epoch_end(self, model):
#         loss = model.get_latest_training_loss()
#         print('Loss after epoch {}: {}'.format(self.epoch, loss))
#         self.epoch += 1

In [7]:
# model = Word2Vec(lists_new, sg=1, size=100, window=5,  min_count=1,  negative=3, sample=0.001, hs=1, workers=10, alpha=0.01, compute_loss=True,callbacks=[callback()])
# model.save("w2v.model")
# model = Word2Vec.load('w2v.model')
# model.train(lists_new,total_examples=len(ppr_list),epochs=20,compute_loss=True,callbacks=[callback()])

In [8]:
data = pd.read_table('/home/duyongkang/GloVe/vectors.txt',header=None)
# data = data.drop([0])
data.columns = ['vector']
data = pd.DataFrame(data.vector.str.split(' ',1).tolist(), columns = ['word','vector'])

In [9]:
data

,word,vector
0,the,-0.861408 -0.628075 0.043010 1.051666 1.215059...
1,of,-0.777877 -0.581980 -0.316073 0.113544 1.49328...
2,and,0.186485 -0.593945 -0.567363 0.008803 1.901995...
3,a,0.297141 -0.679280 -0.371810 0.498104 1.253722...
4,to,0.101414 -1.457081 0.019926 0.647328 1.671532 ...
...,...,...
32212,zrp,0.388301 0.213353 -0.018898 -0.168224 -0.06187...
32213,zs,0.045164 0.112919 0.416550 -0.237265 -0.046351...
32214,zsccmos,-0.080776 0.227994 0.132621 0.081387 -0.221894...
32215,zunic,-0.059311 0.440690 -0.047665 -0.037580 -0.6641...


In [10]:
for i in range(len(data)):
    list_1 = data.vector[i].split(' ')
#     list_1.pop()
    list_1 = [float(x) for x in list_1]
    data.vector[i] = list_1
    if i%5000 == 0:
        print(i)

0
5000
10000
15000
20000
25000
30000


In [11]:
dic = data[['word','vector']].set_index('word').to_dict()['vector']

In [12]:
len(dic['research'])

100

## 计算学者向量

In [13]:
# 合并每个学者的专长
exper_dic = {}   #key为学者编号，value为专长
for i in range(len(auth_list)):
    node = "a"+str(auth_list[i])
    if node not in exper_dic:
        exper_dic[node] = []
    temp = exper_list[i].split()
    exper_dic[node].extend(temp)

In [14]:
#专长向量化
exper_vec = {}

for key in exper_dic:              
    exper_vec[key] = []
    for i in range(100):
        value = 0
        count = 0
        for doc in exper_dic[key]:
            try:
                vec = dic[doc][i]
                value+=vec
            except:
                count+=1
                continue
        value /= (len(exper_dic[key]) - count)
        exper_vec[key].append(value)

In [15]:
# 生成dataframe文件
df_a = pd.DataFrame(pd.Series(exper_vec), columns=['vector'])
df_a = df_a.reset_index().rename(columns = {'index':'node'})

In [16]:
len(df_a.vector[0])

100

## 计算论文向量

In [17]:
#构建一个字典，key为论文编号，value为text
ppr_dic = {}
for i in range(len(ppr_list)):
    node = "p"+str(ppr_list[i])
    if node not in ppr_dic:
        ppr_dic[node] = []
    ppr_dic[node].extend(lists_new[i])

In [18]:
#生成论文的文本特征
ppr_vec = {}
for key in ppr_dic:
    ppr_vec[key] = []
    for i in range(100):
        value = 0
        count = 0
        for doc in ppr_dic[key]:
            try:
                vec = dic[doc][i]
            except:
                count+=1
                continue
            value+=vec
        value /= (len(ppr_dic[key])-count)
        ppr_vec[key].append(value)

In [19]:
# 生成dataframe文件
df_p = pd.DataFrame(pd.Series(ppr_vec), columns=['vector'])
df_p = df_p.reset_index().rename(columns = {'index':'node'})

In [20]:
# 合并作者向量和论文向量
result = df_a.append(df_p)
result.to_pickle('Glove.pkl')